In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import torch

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2
from torchvision.transforms import functional as F

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2
from torchvision.transforms import functional as F

# --- CONFIG ---
ORIG_DIR = "/content/drive/My Drive/mannequin_remover/training_data/orig_data"
MASK_DIR = "/content/drive/My Drive/mannequin_remover/training_data/mask"
SELECTED_FILENAMES = [
    "Damski-pantalon-Rich---Royal-131293202c.jpg",
    "Roklya-Mascara-131614564b.jpg",
    "Roklya-Magic-Nights-131616549c.jpg",
    "Roklya-Magic-Nights-131616549b.jpg",
    "Roklya-Jake-s-131618052b.jpg",
    "Majki-suichar-Armani-Exchange-131625291c.jpg",
    "Majki-pulover-McNeal-131642533b.jpg",
    "Majki-pulover-McNeal-131642533c.jpg",
    "Majka-teniska-Hugo-Boss-131644158b.jpg",
    "Majka-teniska-Hugo-Boss-131644158c.jpg",
    "Roklya-Janina-131914839b.jpg",
    "Roklya-Janina-131914839c.jpg",
    "Damska-bluza-Charles-Vogele-131917814c.jpg",
    "Damska-bluza-Charles-Vogele-131917814b.jpg",
    "Damska-bluza-Laura-Torelli-131918939c.jpg",
    "Damska-bluza-Laura-Torelli-131918939b.jpg",
    "Damska-riza-Unbranded-131922113d.jpg",
    "Damska-bluza-Yessica-131922941c.jpg",
    "Damska-bluza-Yessica-131922941b.jpg",
    "Roklya-Unbranded-131923385c.jpg",
    "Roklya-Unbranded-131923385b.jpg",
    "Damska-riza-Laura-Torelli-131927545b.jpg",
    "Damska-riza-Laura-Torelli-131927545c.jpg",
    "Roklya-S-Oliver-131936460b.jpg",
    "Roklya-S-Oliver-131936460c.jpg",
    "Roklya-Page-One-131941142c.jpg",
    "Roklya-Page-One-131941142b.jpg",
    "Damska-bluza-Clockhouse-131943364c.jpg",
    "Majka-teniska-Livergy-131947217c.jpg",
    "Majka-teniska-Livergy-131947217b.jpg",
    "Majka-riza-Alessandro-Gherardi-132003972b.jpg",
    "Majka-riza-Sonrisa-132006934b.jpg",
    "Damski-kas-pantalon-Chiara-Ferragni-132010631c.jpg",
    "Damski-pantalon-ALESSIA-SANTI-132010813b.jpg",
    "Damski-pantalon-ALESSIA-SANTI-132010813c.jpg",
    "Damski-kas-pantalon-TWINSET-132052617c.jpg",
    "Majki-bokserki-Anko-132361938b.jpg",
    "Majki-bokserki-Anko-132361938c.jpg",
    "Majki-banski-Unbranded-132373417b.jpg",
    "Majki-banski-Unbranded-132373417c.jpg",
    "Majki-banski-Unbranded-132387941c.jpg",
    "Majki-banski-Unbranded-132387941b.jpg",
    "Damski-banski-About-You-132393108c.jpg",
    "Damski-kas-gashterizon-Zara-132396963c.jpg",
    "Damski-kas-gashterizon-Zara-132396963b.jpg",
    "Majki-banski-Sundek-132398175c.jpg",
    "Majki-banski-Sundek-132398175b.jpg",
    "Damsko-belyo-Trendyol-132405322c.jpg",
    "Damski-kas-gashterizon-Mango-132424145c.jpg",
    "Bikini-Adidas-132430176c.jpg",
    "Roklya-Mango-132431513c.jpg",
    "Roklya-Mango-132431513b.jpg",
    "Roklya-Mango-132431597b.jpg",
    "Roklya-America-Today-132431726b.jpg",
    "Roklya-America-Today-132431726c.jpg",
    "Roklya-Style-Cheat-132433975b.jpg",
    "Roklya-Unbranded-132442142b.jpg",
    "Damska-jiletka-Tom-Tailor-132442925b.jpg",
    "Damski-potnik-ONLY-132445567c.jpg",
    "Damski-potnik-ONLY-132445567b.jpg",
    "Damsko-sportno-gornishte-PUMA-132449564c.jpg",
    "Damsko-sportno-gornishte-PUMA-132449564b.jpg",
    "Damsko-sako-H-M-132450375b.jpg",
    "Damsko-palto-Zara-132450631b.jpg",
    "Damsko-yake-Barbara-Lebek-132451625b.jpg",
    "Damska-jiletka-Miss-Chic-132452024c.jpg",
    "Damska-teniska-S-Oliver-Black-Label-132453899b.jpg",
    "Damsko-yake-Zara-132462371b.jpg",
    "Damski-klin-SHEIN-132463151b.jpg",
    "Majki-pantalon-Unbranded-132463406b.jpg",
    "Pola-Lindex-132465412c.jpg",
    "Pola-Lindex-132465412b.jpg",
    "Damsko-sportno-dolnishte-Unbranded-132467183c.jpg",
    "Damsko-sako-United-Colors-Of-Benetton-132467443b.jpg",
    "Majka-teniska-Nike-132141627b.jpg",
    "Damsko-sportno-gornishte-Crane-132469378b.jpg",
    "Damsko-sportno-gornishte-Crane-132469620b.jpg",
    "Majki-suichar-Tommy-Jeans-132469848b.jpg",
    "Damska-bluza-Unbranded-132470119c.jpg",
    "Damska-bluza-Unbranded-132470119b.jpg",
    "Majko-sportno-gornishte-Adidas-132475411b.jpg",
    "Damski-kas-pantalon-Unbranded-132476460c.jpg",
    "Majki-suichar-Unbranded-132477061c.jpg",
    "Majki-suichar-Unbranded-132477061b.jpg",
    "Majko-yake-S-Oliver-132477213b.jpg",
    "Damski-elek-Zara-132477722c.jpg",
    "Damski-elek-Zara-132477722b.jpg",
    "Pola-Sixty-Seven-132477894c.jpg",
    "Pola-Sixty-Seven-132477894b.jpg",
    "Majka-riza-STYLER-132477921c.jpg",
    "Majka-riza-STYLER-132477921b.jpg",
    "Damsko-polo-C-A-132478038c.jpg",
    "Damski-pantalon-H-M-132478105c.jpg",
    "Damska-teniska-UNQ-132478146c.jpg",
    "Majka-bluza-Unbranded-132478511c.jpg",
    "Pola-Remain-132478786c.jpg",
    "Pola-Remain-132478786b.jpg",
    "Damsko-yake-Susan-Graver-132479260b.jpg",
    "Damsko-yake-Susan-Graver-132479260c.jpg",
    "Majki-kas-pantalon-Jack-Wolfskin-132480426b.jpg",
    "Majki-kas-pantalon-Smog-132480654c.jpg",
    "Majki-kas-pantalon-Smog-132480654b.jpg",
    "Majka-teniska-Nordcap-132480675c.jpg",
    "Majka-teniska-Sols-132480941c.jpg",
    "Majki-kas-pantalon-Devred-1902-132481835b.jpg",
    "Majki-kas-pantalon-Devred-1902-132481835c.jpg",
    "Damski-elek-Esprit-132482011c.jpg",
    "Damski-elek-Esprit-132482011b.jpg",
    "Majka-teniska-Nordcap-132482231c.jpg",
    "Majko-sako-Unbranded-132482538c.jpg",
    "Damski-shlifer-ASOS-132482774d.jpg",
    "Majki-potnik-F-F-132483020b.jpg",
    "Majki-potnik-F-F-132483020c.jpg",
    "Majki-pulover-Joop--132483919c.jpg",
    "Damski-kas-pantalon-Unbranded-132484594c.jpg",
    "Damski-kas-pantalon-Unbranded-132484594b.jpg",
    "Poncho-Holly---Whyte-By-Lindex-132484722b.jpg",
    "Poncho-Holly---Whyte-By-Lindex-132484722c.jpg",
    "Damski-kas-pantalon-Miss-Swan-132484819c.jpg",
    "Damski-kas-pantalon-Miss-Swan-132484819b.jpg",
    "Damski-kas-pantalon-Bershka-132484958b.jpg",
    "Poncho-Unbranded-132485027c.jpg",
    "Poncho-Unbranded-132485027b.jpg",
    "Pola-pantalon-Emerald-132485462b.jpg",
    "Pola-pantalon-Emerald-132485462c.jpg",
    "Majki-kas-pantalon-Zara-Man-132485594c.jpg",
    "Majki-kas-pantalon-Zara-Man-132485594b.jpg",
    "Pola-pantalon-Avia-132485972b.jpg",
    "Pola-pantalon-GU-132486150b.jpg",
    "Majko-sako-Rene-Lezard-132486226c.jpg",
    "Majko-yake-BMW-M-Motorsport-x-PUMA-132486771b.jpg",
    "Majko-sportno-gornishte-Adidas-132487331b.jpg",
    "Majki-suichar-Air-Jordan-Nike-132487781b.jpg",
    "Damsko-yake-Hailys-132487787b.jpg",
    "Majko-yake-Saller-132487981b.jpg",
    "Roklya-Adidas-132488058c.jpg",
    "Majka-bluza-Karl-Lagerfeld-132488067c.jpg",
    "Damski-kas-pantalon-Crivit-132488109b.jpg",
    "Damski-kas-pantalon-Crivit-132488109c.jpg",
    "Damski-kas-pantalon-Stradivarius-132488382c.jpg",
    "Damski-elek-Gerry-Weber-132488497c.jpg",
    "Damski-elek-Gerry-Weber-132488497b.jpg",
    "Majki-kas-pantalon-George-132489739b.jpg",
    "Kojena-pola-VILA-132489829b.jpg",
    "Kojena-pola-VILA-132489829c.jpg",
    "Majka-teniska-LC-Waikiki-132490208b.jpg",
    "Majki-elek-Ellesse-132490458b.jpg",
    "Majka-teniska-Crivit-132492108b.jpg",
    "Majka-teniska-Calvin-Klein-132492214c.jpg",
    "Dajdobran-F-F-132493577b.jpg",
    "Majka-bluza-Pull-Bear-132224940b.jpg",
    "Majka-teniska-Adidas-132495419b.jpg",
    "Majka-teniska-Adidas-132495419c.jpg",
    "Damski-banski-Dedoles-132496124c.jpg",
    "Damski-banski-Dedoles-132496124b.jpg",
    "Pola-pantalon-Jones-132496587c.jpg",
    "Poncho-Keawa-132500238b.jpg",
    "Roklya-Adidas-132501117b.jpg",
    "Majko-yake-LCW-132502397c.jpg",
    "Roklya-Laona-131617498b.jpg",
    "Roklya-Laona-131617498c.jpg",
    "Roklya-HEY-KYLA-131621385c.jpg",
     "Damsko-sako-H-M-131341838b.jpg",
    "Roklya-Atmosphere-131346156b.jpg",
    "Damsko-yake-Klass-131341265b.jpg",
    "Damski-pantalon-Peck---Peck-131344931b.jpg",
    "Damski-pantalon-Sinsay-131340769b.jpg",
    "Damski-pantalon-H-M-131341147b.jpg",
    "Detski-suichar-SHEIN-131350407b.jpg",
    "Damska-bluza-Charter-Club-131350333b.jpg",
    "Damsko-sako-Greiff-131346613b.jpg",
    "Damska-bluza-Soprano-131339967b.jpg",
    "Roklya-Unbranded-131347361b.jpg",
    "Damski-pantalon-Heavy-Tools-131339460b.jpg",
    "Roklya-Esmara-131343913b.jpg",
    "Pola-Molly-Jo-131348142b.jpg",
    "Damski-top-Bershka-131339677e.jpg",
    "Damska-riza-Showcase-By-Dorothy-Perkins-131338979b.jpg",
    "Damski-pantalon-Unbranded-131340412b.jpg",
    "Pola-Sandro-131339071b.jpg",
    "Majki-pulover-Galeries-Lafayette-131348208b.jpg",
    "Roklya-Adrom-131340356b.jpg",
    "Damski-potnik-Samoon-131519622b.jpg",
    "Damska-bluza-T-a-t-u--131345648b.jpg",
    "Damsko-yake-Unbranded-131340548b.jpg",
    "Pola-JJXX-131345084b.jpg",
    "Pola-South-131341425b.jpg",
    "Roklya-Unbranded-131350027d.jpg",
    "Roklya-Terre-Bleue-131347332b.jpg",
    "Damska-bluza-Jbc-131340585b.jpg",
    "Roklya-NEW-COLLECTION-130997147b.jpg",
    "Roklya-Atmosphere-131348537b.jpg",
    "Pola-Mango-131339550b.jpg",
    "Damska-bluza-Lena-131345904b.jpg",
    "Damska-jiletka-Colours-Of-The-World-131347353b.jpg",
    "Roklya-Unbranded-131340200b.jpg",
    "Damska-bluza-Unbranded-131350239b.jpg",
    "Damski-top-Unbranded-131340087b.jpg",
    "Damska-bluza-Stitch---Soul-125249655b.jpg",
    "Damski-top-C-A-131339813b.jpg",
    "Damska-bluza-Saint-Tropez-131344639b.jpg",
    "Pola-Sinsay-131348725b.jpg",
    "Roklya-New-Look-131339301b.jpg",
    "Roklya-Zara-131345419b.jpg",
    "Roklya-Zara-Trafaluc-131347275b.jpg",
    "Damsko-palto-Atmosphere-131341326b.jpg",
    "Damska-jiletka-Betty-Barclay-131344849b.jpg",
    "Poncho-Yessica-131343675b.jpg",
    "Damska-bluza---130076974b.jpg",
    "Roklya-Promod-128926082b.jpg",
    "Pola-Hema-131347828b.jpg",
    "Roklya-United-Colors-Of-Benetton-131345490b.jpg",
    "Roklya-Atmosphere-131350393b.jpg",
    "Damski-pantalon-Mango-131155327b.jpg",
    "Damska-bluza-Tally-Weijl-131344954b.jpg",
    "Roklya-R-M-Richards-131342067d.jpg",
    "Roklya-F-F-131347104b.jpg",
    "Pola---pantalon-Zara-131347331b.jpg",
    "Damski-danki-Unbranded-131340081b.jpg",
    "Roklya-Blue-Motion-131343396d.jpg",
    "Roklya-Unbranded-131342584b.jpg",
    "Pola-Esmara-131345552b.jpg",
    "Damsko-yake-Volcano-131340073b.jpg",
    "Roklya-Philosophy-Blues-Original-131348041b.jpg",
    "Roklya-Vero-Moda-130694899b.jpg",
    "Roklya-Mango-131345285b.jpg",
    "Roklya-Etam-131347150b.jpg",
    "Damski-top-Amisu-131339597d.jpg",
    "Damski-pulover-Peter-Hahn-131344158b.jpg",
    "Damski-pulover-Bpc-Bonprix-Collection-131343174b.jpg",
    "Roklya-Body-Flirt-131347186b.jpg",
    "Damska-jiletka-United-Colors-Of-Benetton-131343503b.jpg",
    "Roklya-Trendyol-131338953b.jpg",
    "Roklya-Party-131347587b.jpg",
    "Damski-kas-pantalon-Barbie-131347501b.jpg",
    "Roklya-H-M-131347068b.jpg",
    "Majko-yake-Bugatti-131345379c.jpg",
    "Damska-jiletka-Blend-She-131343640b.jpg",
    "Damska-jiletka-Atmosphere-131341853b.jpg",
    "Roklya-Day-Birger-Et-Mikkelsen-130920457b.jpg",
    "Roklya-Bpc-Bonprix-Collection-131343369b.jpg",
    "Pola-Esprit-131339283b.jpg",
    "Pola-Esmara-131344032b.jpg",
    "Roklya-NEW-COLLECTION-131338691b.jpg",
    "Roklya-Zee-Lane-131333791b.jpg",
    "Damski-pantalon-For-Women-131340870b.jpg",
    "Damska-bluza-Ellen-Amber-131332281b.jpg",
    "Roklya-Pull-Bear-131335015b.jpg",
    "Majki-pantalon-LC-Waikiki-131337169b.jpg",
    "Damski-top-H-M-131334438b.jpg",
    "Pola-Manguun-131345231b.jpg",
    "Roklya-Pull-Bear-131334807b.jpg",
    "Pola-Trend-One-131332647b.jpg",
    "Damsko-palto-George-131336223b.jpg",
    "Roklya-Mango-131335212b.jpg",
    "Roklya-Gina-Tricot-131332712b.jpg",
    "Roklya-Mango-131335170c.jpg",
    "Damski-danki-Unbranded-131334182a.jpg",
    "Damska-jiletka-Unbranded-131337971b.jpg",
    "Damska-bluza-Unbranded-131333930b.jpg",
    "Roklya-Unbranded-131332808b.jpg"
]

BATCH_SIZE = 16
EPOCHS = 15
LEARNING_RATE = 1e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_existing_pairs(orig_dir, mask_dir, filenames):
    valid = []
    for fname in filenames:
        orig_path = os.path.join(orig_dir, fname)
        mask_name = fname.replace('.jpg', '_mask.png')
        mask_path = os.path.join(mask_dir, mask_name)
        if os.path.isfile(orig_path) and os.path.isfile(mask_path):
            valid.append(fname)
        else:
            print(f"HIÁNYZIK: {fname} vagy a maszkja")
    return valid


# --- DATASET ---
class SegmentationDataset(Dataset):
    def __init__(self, orig_dir, mask_dir, filelist, transforms=None):
        self.orig_dir = orig_dir
        self.mask_dir = mask_dir
        self.filelist = filelist
        self.transforms = transforms

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        img_name = self.filelist[idx]
        img_path = os.path.join(self.orig_dir, img_name)
        mask_name = img_name.replace('.jpg', '_mask.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        mask = np.array(mask)
        mask = (mask > 0).astype(np.uint8)
        mask = torch.as_tensor(mask, dtype=torch.uint8)
        target = {}
        # Assume one object per image (entire clothing region as one mask)
        pos = np.where(mask)
        if len(pos[0]) == 0:
            # If no mask present, return dummy target
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
            target["masks"] = torch.zeros((0, mask.shape[0], mask.shape[1]), dtype=torch.uint8)
        else:
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes = torch.as_tensor([[xmin, ymin, xmax, ymax]], dtype=torch.float32)
            target["boxes"] = boxes
            target["labels"] = torch.ones((1,), dtype=torch.int64)  # all foreground
            target["masks"] = mask.unsqueeze(0)
        target["image_id"] = torch.tensor([idx])
        return image, target

# --- DICE & IOU ---
def dice_score(pred, target):
    smooth = 1.0
    pred = pred.bool()
    target = target.bool()
    intersection = (pred & target).sum().float()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

def iou_score(pred, target):
    smooth = 1.0
    pred = pred.bool()
    target = target.bool()
    intersection = (pred & target).sum().float()
    union = (pred | target).sum().float()
    return (intersection + smooth) / (union + smooth)

# --- TRAINING LOOP ---
def evaluate(model, loader):
    model.eval()
    dice_scores, iou_scores = [], []
    with torch.no_grad():
        for images, targets in loader:
            images = list(img.to(DEVICE) for img in images)
            outputs = model(images)
            for out, tgt in zip(outputs, targets):
                if len(out['masks']) == 0:
                    pred_mask = torch.zeros_like(tgt['masks'][0])
                else:
                    pred_mask = (out['masks'][0,0] > 0.5).cpu()
                dice = dice_score(pred_mask, tgt['masks'][0])
                iou = iou_score(pred_mask, tgt['masks'][0])
                dice_scores.append(dice.item())
                iou_scores.append(iou.item())
    return np.mean(dice_scores), np.mean(iou_scores)

def collate_fn(batch):
    return tuple(zip(*batch))

dataset = SegmentationDataset(ORIG_DIR, MASK_DIR, SELECTED_FILENAMES)

In [ ]:
n_total = len(dataset)
test_len = int(0.1 * n_total)
val_len = int(0.1 * n_total)
train_len = n_total - val_len - test_len

# Fix order for reproducibility
train_set, val_set, test_set = random_split(
    dataset, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_set, batch_size=1, shuffle=True, collate_fn=collate_fn)
test_loader  = DataLoader(test_set, batch_size=1, shuffle=True, collate_fn=collate_fn)

# Birefnet

In [ ]:
!pip install kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForImageSegmentation
import torch

birefnet = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_model():
    """
    BiRefNet modell betöltése memóriába, ha még nincs betöltve
    """
    global birefnet
    if birefnet is None:
        try:
            # Explicit model_type megadása a config.json hiánya miatt
            birefnet = AutoModelForImageSegmentation.from_pretrained(
                "zhengpeng7/BiRefNet",
                trust_remote_code=True,
                config={'model_type': 'custom_segmentation_model'}
            )
            birefnet.to(device)
            birefnet.eval()
            if device.type == 'cuda':
                birefnet.half()
        except Exception as e:
            print(f"Hiba a modell betöltésekor: {e}")
            raise e
    return birefnet

birefnet = load_model()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

BiRefNet_config.py:   0%|          | 0.00/298 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhengpeng7/BiRefNet:
- BiRefNet_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


birefnet.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhengpeng7/BiRefNet:
- birefnet.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


model.safetensors:   0%|          | 0.00/444M [00:00<?, ?B/s]

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torchvision.transforms.functional as TF

class SegmentationDataset(Dataset):
    def __init__(self, orig_dir, mask_dir, filelist, image_size=512, transforms=None):
        self.orig_dir = orig_dir
        self.mask_dir = mask_dir
        self.filelist = filelist
        self.image_size = image_size
        self.transforms = transforms

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        img_name = self.filelist[idx]
        img_path = os.path.join(self.orig_dir, img_name)
        mask_name = img_name.replace('.jpg', '_mask.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = Image.open(img_path).convert("RGB").resize((self.image_size, self.image_size))
        mask = Image.open(mask_path).convert("L").resize((self.image_size, self.image_size))
        if self.transforms:
            image = self.transforms(image)
        else:
            image = F.to_tensor(image)
        mask = np.array(mask)
        mask = (mask > 0).astype(np.float32)
        mask = torch.as_tensor(mask, dtype=torch.float32)
        return image, mask


In [ ]:
def dice_score(pred, target):
    smooth = 1.0
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

def iou_score(pred, target):
    smooth = 1.0
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + smooth) / (union + smooth)


In [ ]:
def evaluate_birefnet(model, loader, device):
    model.eval()
    dice_scores = []
    iou_scores = []
    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device)
            masks = masks.to(device)
            # Forward
            logits = model(images)
            # BiRefNet több outputot is adhat, ha tuple: csak az első kell!
            if isinstance(logits, tuple):
                logits = logits[0]
            preds = torch.sigmoid(logits)
            for pred, gt in zip(preds, masks):
                dice = dice_score(pred, gt)
                iou = iou_score(pred, gt)
                dice_scores.append(dice.item())
                iou_scores.append(iou.item())
    return np.mean(dice_scores), np.mean(iou_scores)


In [ ]:
class SimpleSegmentationDataset(Dataset):
    def __init__(self, orig_dir, mask_dir, filelist, image_size=512):
        self.orig_dir = orig_dir
        self.mask_dir = mask_dir
        self.filelist = filelist
        self.image_size = image_size

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, idx):
        img_name = self.filelist[idx]
        img_path = os.path.join(self.orig_dir, img_name)
        mask_name = img_name.replace('.jpg', '_mask.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = Image.open(img_path).convert("RGB").resize((self.image_size, self.image_size))
        mask = Image.open(mask_path).convert("L").resize((self.image_size, self.image_size))
        from torchvision.transforms import functional as TF
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask).float().squeeze(0)
        mask = (mask > 0).float()
        return image, mask


In [ ]:
def evaluate_birefnet(model, loader, device):
    model.eval()
    dice_scores = []
    iou_scores = []

    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device).float()
            masks = masks.to(device).float()

            # ✅ Debug the evaluation output structure
            logits = model(images)

            print("=== EVALUATION MODE DEBUG ===")
            print("logits type:", type(logits))

            if isinstance(logits, list):
                print("logits length:", len(logits))
                for i, item in enumerate(logits):
                    print(f"  Item {i}: type={type(item)}")
                    if isinstance(item, tuple):
                        print(f"    Tuple length: {len(item)}")
                        for j, sub_item in enumerate(item):
                            if hasattr(sub_item, 'shape'):
                                print(f"      [{j}] shape: {sub_item.shape}")
                            else:
                                print(f"      [{j}] type: {type(sub_item)}")
                    elif hasattr(item, 'shape'):
                        print(f"    Shape: {item.shape}")

            # ✅ Try multiple extraction strategies
            extracted_logits = None

            # Strategy 1: Same as training
            try:
                if isinstance(logits, list) and len(logits) >= 1:
                    main_outputs = logits[0]
                    if isinstance(main_outputs, tuple) and len(main_outputs) >= 2:
                        final_outputs = main_outputs[1]
                        extracted_logits = final_outputs[-1]
                        print("✓ Strategy 1 worked (same as training)")
            except:
                print("✗ Strategy 1 failed")

            # Strategy 2: Direct list access
            if extracted_logits is None:
                try:
                    if isinstance(logits, list):
                        # Try to find the 512x512 tensor directly
                        for item in logits:
                            if hasattr(item, 'shape') and item.shape[-2:] == (512, 512):
                                extracted_logits = item
                                print("✓ Strategy 2 worked (direct 512x512 search)")
                                break
                except:
                    print("✗ Strategy 2 failed")

            # Strategy 3: Take the last item if it's a tensor
            if extracted_logits is None:
                try:
                    if isinstance(logits, list) and len(logits) > 0:
                        last_item = logits[-1]
                        if hasattr(last_item, 'shape'):
                            extracted_logits = last_item
                            print("✓ Strategy 3 worked (last item)")
                except:
                    print("✗ Strategy 3 failed")

            if extracted_logits is None:
                print("❌ All strategies failed!")
                return 0.0, 0.0  # Return dummy values to avoid crash

            print(f"Final extracted logits shape: {extracted_logits.shape}")

            # Convert to probabilities
            preds = torch.sigmoid(extracted_logits)

            # Calculate metrics for each sample in the batch
            for pred, gt in zip(preds, masks.unsqueeze(1)):
                dice = dice_score(pred, gt)
                iou = iou_score(pred, gt)
                dice_scores.append(dice)
                iou_scores.append(iou)

            # Only debug the first batch
            break

    model.train()  # Switch back to training mode
    return np.mean(dice_scores), np.mean(iou_scores)

In [ ]:
import torch.optim as optim
from torch.utils.data import random_split
import numpy as np

def dice_score(pred, target, smooth=1e-6):
    """Calculate Dice score for tensors on any device"""
    pred = pred.to(target.device)
    pred_flat = pred.view(-1)
    target_flat = target.view(-1)
    intersection = (pred_flat * target_flat).sum()
    dice = (2. * intersection + smooth) / (pred_flat.sum() + target_flat.sum() + smooth)
    return dice.item()

def iou_score(pred, target, smooth=1e-6):
    """Calculate IoU score for tensors on any device"""
    pred = pred.to(target.device)
    pred_flat = pred.view(-1)
    target_flat = target.view(-1)
    intersection = (pred_flat * target_flat).sum()
    union = pred_flat.sum() + target_flat.sum() - intersection
    iou = (intersection + smooth) / (union + smooth)
    return iou.item()

def extract_birefnet_output(logits):
    """Extract the 512x512 output from BiRefNet's complex nested structure."""
    # Strategy 1: Training mode structure [main_outputs, aux_outputs]
    if isinstance(logits, list) and len(logits) >= 1:
        main_outputs = logits[0]
        if isinstance(main_outputs, tuple) and len(main_outputs) >= 2:
            final_outputs = main_outputs[1]
            if isinstance(final_outputs, list) and len(final_outputs) > 0:
                return final_outputs[-1]  # 512x512 output

    # Strategy 2: Find any 512x512 tensor in the structure
    def find_512x512_tensor(obj):
        if hasattr(obj, 'shape') and len(obj.shape) >= 2 and obj.shape[-2:] == (512, 512):
            return obj
        elif isinstance(obj, (list, tuple)):
            for item in obj:
                result = find_512x512_tensor(item)
                if result is not None:
                    return result
        return None

    result = find_512x512_tensor(logits)
    if result is not None:
        return result

    # Strategy 3: Take the last tensor we can find
    def find_any_tensor(obj):
        if hasattr(obj, 'shape'):
            return obj
        elif isinstance(obj, (list, tuple)):
            for item in reversed(obj):  # Try from the end
                result = find_any_tensor(item)
                if result is not None:
                    return result
        return None

    return find_any_tensor(logits)

def evaluate_birefnet(model, loader, device):
    model.eval()
    dice_scores = []
    iou_scores = []

    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device).float()
            masks = masks.to(device).float()

            logits = model(images)
            extracted_logits = extract_birefnet_output(logits)

            if extracted_logits is None:
                print("Warning: Could not extract valid output from BiRefNet")
                continue

            # Convert to probabilities
            preds = torch.sigmoid(extracted_logits)

            # Calculate metrics for each sample in the batch
            for pred, gt in zip(preds, masks.unsqueeze(1)):
                dice = dice_score(pred, gt)
                iou = iou_score(pred, gt)
                dice_scores.append(dice)
                iou_scores.append(iou)

    model.train()
    return np.mean(dice_scores), np.mean(iou_scores)

# --- Parameters ---
BATCH_SIZE = 2
EPOCHS = 200
LEARNING_RATE = 3e-5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMAGE_SIZE = 512

# --- Data split & Loader ---
dataset = SimpleSegmentationDataset(ORIG_DIR, MASK_DIR, SELECTED_FILENAMES, image_size=IMAGE_SIZE)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, drop_last=True)

# --- Model, loss, optimizer ---
model = birefnet.float()
model.to(DEVICE)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images = images.to(DEVICE).float()
        masks = masks.to(DEVICE).float()

        optimizer.zero_grad()

        logits = model(images)
        extracted_logits = extract_birefnet_output(logits)

        if extracted_logits is None:
            print("Error: Could not extract valid output from BiRefNet")
            continue

        loss = loss_fn(extracted_logits, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # --- Evaluate ---
    train_dice, train_iou = evaluate_birefnet(model, train_loader, DEVICE)
    val_dice, val_iou = evaluate_birefnet(model, val_loader, DEVICE)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {epoch_loss/len(train_loader):.4f} | "
          f"Train Dice: {train_dice:.4f} | Train IoU: {train_iou:.4f} | "
          f"Val Dice: {val_dice:.4f} | Val IoU: {val_iou:.4f}")

Epoch 1/200 | Loss: 0.0357 | Train Dice: 0.8757 | Train IoU: 0.7992 | Val Dice: 0.8737 | Val IoU: 0.7915
Epoch 2/200 | Loss: 0.0251 | Train Dice: 0.9004 | Train IoU: 0.8355 | Val Dice: 0.8998 | Val IoU: 0.8301
Epoch 3/200 | Loss: 0.0211 | Train Dice: 0.9007 | Train IoU: 0.8355 | Val Dice: 0.9028 | Val IoU: 0.8323


KeyboardInterrupt: 

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images = images.to(DEVICE).float()
        masks = masks.to(DEVICE).float()

        optimizer.zero_grad()

        logits = model(images)
        extracted_logits = extract_birefnet_output(logits)

        if extracted_logits is None:
            print("Error: Could not extract valid output from BiRefNet")
            continue

        loss = loss_fn(extracted_logits, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # --- Evaluate ---
    train_dice, train_iou = evaluate_birefnet(model, train_loader, DEVICE)
    val_dice, val_iou = evaluate_birefnet(model, val_loader, DEVICE)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {epoch_loss/len(train_loader):.4f} | "
          f"Train Dice: {train_dice:.4f} | Train IoU: {train_iou:.4f} | "
          f"Val Dice: {val_dice:.4f} | Val IoU: {val_iou:.4f}")

Epoch 1/200 | Loss: 0.0201 | Train Dice: 0.9153 | Train IoU: 0.8581 | Val Dice: 0.9148 | Val IoU: 0.8524
Epoch 2/200 | Loss: 0.0175 | Train Dice: 0.9127 | Train IoU: 0.8529 | Val Dice: 0.9085 | Val IoU: 0.8409
Epoch 3/200 | Loss: 0.0144 | Train Dice: 0.9175 | Train IoU: 0.8600 | Val Dice: 0.9125 | Val IoU: 0.8468
Epoch 4/200 | Loss: 0.0148 | Train Dice: 0.9131 | Train IoU: 0.8525 | Val Dice: 0.9064 | Val IoU: 0.8374
Epoch 5/200 | Loss: 0.0137 | Train Dice: 0.9157 | Train IoU: 0.8570 | Val Dice: 0.9104 | Val IoU: 0.8433
Epoch 6/200 | Loss: 0.0140 | Train Dice: 0.9233 | Train IoU: 0.8703 | Val Dice: 0.9161 | Val IoU: 0.8527
Epoch 7/200 | Loss: 0.0187 | Train Dice: 0.8655 | Train IoU: 0.7834 | Val Dice: 0.8595 | Val IoU: 0.7696
Epoch 8/200 | Loss: 0.0157 | Train Dice: 0.9129 | Train IoU: 0.8541 | Val Dice: 0.9005 | Val IoU: 0.8288
Epoch 9/200 | Loss: 0.0162 | Train Dice: 0.9126 | Train IoU: 0.8521 | Val Dice: 0.9047 | Val IoU: 0.8345
Epoch 10/200 | Loss: 0.0143 | Train Dice: 0.9251 | Trai

KeyboardInterrupt: 

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images = images.to(DEVICE).float()
        masks = masks.to(DEVICE).float()

        optimizer.zero_grad()

        logits = model(images)
        extracted_logits = extract_birefnet_output(logits)

        if extracted_logits is None:
            print("Error: Could not extract valid output from BiRefNet")
            continue

        loss = loss_fn(extracted_logits, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # --- Evaluate ---
    train_dice, train_iou = evaluate_birefnet(model, train_loader, DEVICE)
    val_dice, val_iou = evaluate_birefnet(model, val_loader, DEVICE)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {epoch_loss/len(train_loader):.4f} | "
          f"Train Dice: {train_dice:.4f} | Train IoU: {train_iou:.4f} | "
          f"Val Dice: {val_dice:.4f} | Val IoU: {val_iou:.4f}")

Epoch 1/200 | Loss: 0.0110 | Train Dice: 0.9314 | Train IoU: 0.8791 | Val Dice: 0.8973 | Val IoU: 0.8259
Epoch 2/200 | Loss: 0.0103 | Train Dice: 0.9380 | Train IoU: 0.8893 | Val Dice: 0.9098 | Val IoU: 0.8445
Epoch 3/200 | Loss: 0.0109 | Train Dice: 0.9360 | Train IoU: 0.8901 | Val Dice: 0.9168 | Val IoU: 0.8554
Epoch 4/200 | Loss: 0.0101 | Train Dice: 0.9401 | Train IoU: 0.8942 | Val Dice: 0.9143 | Val IoU: 0.8511
Epoch 5/200 | Loss: 0.0096 | Train Dice: 0.9346 | Train IoU: 0.8849 | Val Dice: 0.9025 | Val IoU: 0.8346
Epoch 6/200 | Loss: 0.0096 | Train Dice: 0.9371 | Train IoU: 0.8897 | Val Dice: 0.9115 | Val IoU: 0.8467
Epoch 7/200 | Loss: 0.0107 | Train Dice: 0.9417 | Train IoU: 0.8958 | Val Dice: 0.9163 | Val IoU: 0.8534
Epoch 8/200 | Loss: 0.0100 | Train Dice: 0.9426 | Train IoU: 0.8990 | Val Dice: 0.9182 | Val IoU: 0.8578
Epoch 9/200 | Loss: 0.0099 | Train Dice: 0.9382 | Train IoU: 0.8923 | Val Dice: 0.9014 | Val IoU: 0.8347
Epoch 10/200 | Loss: 0.0099 | Train Dice: 0.9341 | Trai

KeyboardInterrupt: 

In [ ]:

from torchvision import transforms
from torchvision.transforms import functional as TF
import matplotlib.pyplot as plt
import numpy as np

def show_validation_preds_overlay(model, val_loader):
    model.eval()
    resize_to = (992, 992)
    with torch.no_grad():
        for idx, (images, targets) in enumerate(val_loader):
            image = images[0]
            if hasattr(image, 'cpu'):  # torch tensor
                image_pil = transforms.ToPILImage()(image.cpu())
            else:  # PIL Image
                image_pil = image
            image_pil = TF.resize(image_pil, resize_to)
            image = TF.to_tensor(image_pil).to(DEVICE)

            output = model(image.unsqueeze(0))
            img_np = image.cpu().permute(1,2,0).numpy().astype(np.float32)
            try:
                mask = output[0]
                mask_sigmoid = torch.sigmoid(mask[0, 0])
                pred_mask = (mask_sigmoid > 0.1).cpu().numpy().astype(np.float32)
            except Exception as e:
                print(f"Hiba a mask extractnál: {e}")
                pred_mask = np.zeros((img_np.shape[0], img_np.shape[1]), dtype=np.uint8)
            fig, ax = plt.subplots(figsize=(5,5))
            ax.imshow(img_np)
            ax.imshow(pred_mask, alpha=0.4, cmap='Reds')
            ax.set_title('Prediktált maszk overlay')
            ax.axis('off')
            plt.show()


In [ ]:
show_validation_preds_overlay(model, test_loader)

RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [ ]:
show_validation_preds_overlay(model, val_loader)

# Save artifacts

In [ ]:
import datetime
import torch
import os
import json
import datetime

filtered_filenames = get_existing_pairs(ORIG_DIR, MASK_DIR, SELECTED_FILENAMES)

def save_artifacts(
    model, optimizer, epoch, best_val_metrics,
    selected_filenames, splits, train_logs, config, save_dir
):
    os.makedirs(save_dir, exist_ok=True)
    # 1. Modell és optimizer checkpoint
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'best_val_metrics': best_val_metrics,
        'config': config
    }, os.path.join(save_dir, "checkpoint.pt"))

    # 2. Fájllisták és split indexek
    with open(os.path.join(save_dir, "selected_filenames.json"), "w") as f:
        json.dump(selected_filenames, f, indent=2)
    with open(os.path.join(save_dir, "split_indices.json"), "w") as f:
        json.dump(splits, f, indent=2)
    # 3. Tréning log
    #with open(os.path.join(save_dir, "train_logs.json"), "w") as f:
    #    json.dump(train_logs, f, indent=2)
    print(f"Mentés kész: {save_dir}")

In [ ]:


now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
SAVE_DIR = f"/content/drive/My Drive/birefnet_mannequin_remover_artifacts/{now}"
splits = {
    "train_indices": train_set.indices,
    "val_indices": val_set.indices,
    "test_indices": test_set.indices,
}
config = {
    "BATCH_SIZE": BATCH_SIZE,
    "EPOCHS": EPOCHS,
    "LEARNING_RATE": LEARNING_RATE,
    "model_type": "maskrcnn_resnet50_fpn_v2",
    "orig_dir": ORIG_DIR,
    "mask_dir": MASK_DIR
}
train_logs = [] # vagy ami van

save_artifacts(
    model, optimizer, epoch+1,
    {"val_dice": val_dice, "val_iou": val_iou},
    filtered_filenames, splits, train_logs, config, SAVE_DIR
)

Mentés kész: /content/drive/My Drive/birefnet_mannequin_remover_artifacts/20250703_190222


# Import model from artifacts

In [ ]:
checkpoint = torch.load("/content/drive/My Drive/birefnet_mannequin_remover_artifacts/20250703_190222/checkpoint.pt", map_location="cuda", weights_only=False)

In [ ]:
birefnet = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_model():
    """
    BiRefNet modell betöltése memóriába, ha még nincs betöltve
    """
    global birefnet
    if birefnet is None:
        try:
            # Explicit model_type megadása a config.json hiánya miatt
            birefnet = AutoModelForImageSegmentation.from_pretrained(
                "zhengpeng7/BiRefNet",
                trust_remote_code=True,
                config={'model_type': 'custom_segmentation_model'}
            )
            birefnet.to(device)
            birefnet.eval()
            if device.type == 'cuda':
                birefnet.half()
        except Exception as e:
            print(f"Hiba a modell betöltésekor: {e}")
            raise e
    return birefnet

birefnet = load_model()

Hiba a modell betöltésekor: name 'AutoModelForImageSegmentation' is not defined


NameError: name 'AutoModelForImageSegmentation' is not defined

In [ ]:
!pip install kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForImageSegmentation
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Modell betöltése
birefnet = AutoModelForImageSegmentation.from_pretrained(
    "zhengpeng7/BiRefNet",
    trust_remote_code=True,
    config={'model_type': 'custom_segmentation_model'}
)
birefnet.to(device)
birefnet.eval()
if device.type == 'cuda':
    birefnet.half()

# 2. Saját checkpoint betöltése
checkpoint = torch.load(
    "/content/drive/My Drive/birefnet_mannequin_remover_artifacts/20250703_190222/checkpoint.pt",
    map_location=device,
    weights_only=False
)

# 3. Saját tanított súlyok visszatöltése
birefnet.load_state_dict(checkpoint['model_state_dict'])

model = birefnet

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


model.safetensors:   0%|          | 0.00/444M [00:00<?, ?B/s]

In [ ]:

from torchvision import transforms
from torchvision.transforms import functional as TF
import matplotlib.pyplot as plt
import numpy as np

def show_validation_preds_overlay(model, val_loader):
    model.eval()
    resize_to = (992, 992)
    with torch.no_grad():
        for idx, (images, targets) in enumerate(val_loader):
            image = images[0]
            if hasattr(image, 'cpu'):  # torch tensor
                image_pil = transforms.ToPILImage()(image.cpu())
            else:  # PIL Image
                image_pil = image
            image_pil = TF.resize(image_pil, resize_to)
            image = TF.to_tensor(image_pil).to(DEVICE)
            # Match dtype to model weights
            model_dtype = next(model.parameters()).dtype
            image = image.to(model_dtype)

            output = model(image.unsqueeze(0))
            img_np = image.cpu().permute(1,2,0).numpy().astype(np.float32)
            try:
                mask = output[0]
                mask_sigmoid = torch.sigmoid(mask[0, 0])
                pred_mask = (mask_sigmoid > 0.1).cpu().numpy().astype(np.float32)
            except Exception as e:
                print(f"Hiba a mask extractnál: {e}")
                pred_mask = np.zeros((img_np.shape[0], img_np.shape[1]), dtype=np.uint8)
            fig, ax = plt.subplots(figsize=(5,5))
            ax.imshow(img_np)
            ax.imshow(pred_mask, alpha=0.4, cmap='Reds')
            ax.set_title('Prediktált maszk overlay')
            ax.axis('off')
            plt.show()


In [ ]:
show_validation_preds_overlay(model, test_loader)